In [1]:
import numpy as np
import networkx as nx
from collections import Counter

In [2]:
test_input = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
..........."""

In [3]:
input = open("inputs/21").read()

In [4]:
def parse_board(input):
    return np.array(list(map(list, input.splitlines())))

In [5]:
# board = parse_board(test_input)
board = parse_board(input)

In [6]:
board.shape

(131, 131)

In [7]:
start = tuple(int(i) for i in np.array(board.shape) // 2)
start

(65, 65)

In [8]:
import math

math.prod(board.shape) - (board == "#").sum()

np.int64(15091)

In [9]:
Counter(board.flatten())

Counter({np.str_('.'): 15090, np.str_('#'): 2070, np.str_('S'): 1})

In [10]:
directions = {"right": (0, 1), "left": (0, -1), "up": (-1, 0), "down": (1, 0)}

directions = list(directions.values())
directions

[(0, 1), (0, -1), (-1, 0), (1, 0)]

In [11]:
from functools import lru_cache


def in_bounds(shape, pos):
    return all(0 <= pos[i] < shape[i] for i in range(len(shape)))


def add_tuples(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))


@lru_cache(None)
def walk(pos, steps_remaining):
    if not in_bounds(board.shape, pos) or board[pos] == "#":
        return frozenset()

    if steps_remaining == 0:
        return frozenset([pos])

    return frozenset.union(
        *(
            walk(add_tuples(pos, direction), steps_remaining - 1)
            for direction in directions
        )
    )

In [12]:
len(walk(start, 64))

3709

In [13]:
walk.cache_info()

CacheInfo(hits=214147, misses=93298, maxsize=None, currsize=93298)

In [15]:
import heapq
import numpy as np
from collections import defaultdict


def in_bounds(shape, pos):
    return all(0 <= pos[i] < shape[i] for i in range(len(shape)))


def add_tuples(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))


def mod_tuple(tuple1, tuple2):
    return tuple(map(lambda x, shape: x % shape, tuple1, tuple2))


def get_possible_neighbors(pos):
    for new_pos in [add_tuples(pos, dir) for dir in directions]:
        new_pos_mod = mod_tuple(new_pos, board.shape)
        if board[new_pos_mod] != "#":
            yield new_pos


pq = [(start, 0)]
distances = defaultdict(lambda: np.inf)
distances[start] = 0
heapq.heapify(pq)

# MAX_DISTANCE = 3000
MAX_DISTANCE = 1000

while pq:
    current_node, current_distance = heapq.heappop(pq)

    # if we've already found a better path skip
    # this relates to the inability to update the priorities in the heapq
    if distances[current_node] < current_distance:
        continue

    if current_distance > MAX_DISTANCE:
        continue

    for n in get_possible_neighbors(current_node):
        if current_distance + 1 < distances[n]:
            distances[n] = current_distance + 1
            heapq.heappush(pq, (n, current_distance + 1))

In [16]:
len(distances)

1764411

In [17]:
import pandas as pd

all_values = pd.DataFrame(distances.values(), columns=["dist"])
all_values

,dist
0,0
1,1
2,1
3,1
4,1
...,...
1764406,1001
1764407,1001
1764408,1000
1764409,1001


In [18]:
import pandas as pd
import numpy as np


def count_values_leq_k(series):
    """
    For each value k from min to max in the series, count:
    1. How many values are <= k
    2. How many values are <= k AND have the same parity as k

    Parameters:
    series (pd.Series): A series of positive integers

    Returns:
    pd.DataFrame: DataFrame with columns 'k', 'count_leq', and 'count_leq_same_parity'
    """
    # Get unique values and their counts
    value_counts = series.value_counts().sort_index()

    # Create range from min to max
    k_range = np.arange(value_counts.index.min(), value_counts.index.max() + 1)

    # Calculate cumulative sum of all counts
    cumulative_counts = pd.Series(0, index=k_range)
    cumulative_counts[value_counts.index] = value_counts
    cumulative_counts = cumulative_counts.cumsum()

    # Split the series into even and odd numbers
    even_counts = series[series % 2 == 0].value_counts().sort_index()
    odd_counts = series[series % 2 == 1].value_counts().sort_index()

    # Calculate cumulative sums for even and odd numbers
    even_cumulative = pd.Series(0, index=k_range)
    odd_cumulative = pd.Series(0, index=k_range)

    even_cumulative[even_counts.index] = even_counts
    odd_cumulative[odd_counts.index] = odd_counts

    even_cumulative = even_cumulative.cumsum()
    odd_cumulative = odd_cumulative.cumsum()

    # Create result DataFrame
    result = pd.DataFrame(
        {
            "k": k_range,
            "count_leq": cumulative_counts.values,
            "count_leq_same_parity": np.where(
                k_range % 2 == 0, even_cumulative.values, odd_cumulative.values
            ),
        }
    )

    return result


step_df = count_values_leq_k(all_values["dist"])
step_df

,k,count_leq,count_leq_same_parity
0,0,1,1
1,1,5,4
2,2,12,8
3,3,22,14
4,4,36,22
...,...,...,...
997,997,1750372,876041
998,998,1753766,877725
999,999,1757369,879644
1000,1000,1760992,881348


In [19]:
sum((v <= 15) for v in distances.values())

416

In [20]:
num_steps = 64
sum((v <= num_steps) and (v % 2 == num_steps % 2) for v in distances.values())

3709

In [21]:
step_df.iloc[64]

k                          64
count_leq                7268
count_leq_same_parity    3709
Name: 64, dtype: int64

In [22]:
board.shape

(131, 131)

In [23]:
points_to_fit = step_df.iloc[[65] + [65 + board.shape[0] * i for i in range(1, 8)]]
points_to_fit

,k,count_leq,count_leq_same_parity
65,65,7528,3819
196,196,67868,34099
327,327,188548,94549
458,458,369568,185169
589,589,610928,305959
720,720,912628,456919
851,851,1274668,638049
982,982,1697048,849349


In [24]:
import numpy as np
from scipy.interpolate import lagrange

X = [1, 2, 3]
y = points_to_fit["count_leq_same_parity"].values[:3]

# Fit Lagrange polynomial
poly = lagrange(X, y)

print(poly)

           2
1.508e+04 x - 1.498e+04 x + 3709


In [25]:
# steps_to_take = 196
# steps_to_take = 327
# steps_to_take = 982
steps_to_take = 26501365
n = (steps_to_take - 65) // board.shape[0] + 1
n

202301

In [26]:
int(poly(n))

617361073602319